Source of datasets https://data.gov.uk/dataset/176ae264-2484-4afe-a297-d51798eb8228/gp-practice-prescribing-data-presentation-level

In [174]:
import numpy as np
import pandas as pd
import re
import os
import gc
%matplotlib inline

In [175]:
prescription_datafile = "extracted/2018_10_Oct/T201810PDPI BNFT.CSV"
prescription_df = pd.read_csv(prescription_datafile)
prescription_df.columns = [col.strip() for col in prescription_df.columns]

In [176]:
prescription_df.head()

,SHA,PCT,PRACTICE,BNF CODE,BNF NAME,ITEMS,NIC,ACT COST,QUANTITY,PERIOD,
0,Q44,RBL,Y00493,0501011P0AAAFAF,Phenoxymethylpenicillin_Soln 250mg/5ml,1,7.28,6.77,100,201810,
1,Q44,RTV,Y04937,0304010W0AAADAD,Promethazine HCl_Tab 10mg,1,1.48,1.49,28,201810,
2,Q44,RTV,Y04937,0401010Y0AAABAB,Zolpidem Tart_Tab 10mg,1,0.49,0.57,14,201810,
3,Q44,RTV,Y04937,0401010Z0AAAAAA,Zopiclone_Tab 7.5mg,3,0.66,0.95,21,201810,
4,Q44,RTV,Y04937,0401020B0AAAAAA,Buspirone HCl_Tab 5mg,1,5.77,5.47,42,201810,


Firstly we have to get a list of all the non generic drugs. The pattern for BNF codes Is referenced here https://ebmdatalab.net/prescribing-data-bnf-codes/
 In principle anything that has AA in the 10 and 11 position of the BNF code is a generic. And using the last two characters gives a pointer to the generic of any branded drugs.

This should give us a dict with the Generic BNF code as the key and value contains an array of the discovered branded BNF codes.

In [177]:
prescription_df['bnf'] = prescription_df['BNF CODE'].str.slice(0, 9)
prescription_df['product'] = prescription_df['BNF CODE'].str.slice(9, 11)
prescription_df['strength'] = prescription_df['BNF CODE'].str.slice(11, 13)
prescription_df['generic_code'] = prescription_df['BNF CODE'].str.slice(13, 15)
prescription_df['GENERIC BNF'] = (
    prescription_df['bnf'] + 'AA' +
    prescription_df['generic_code'] +
    prescription_df['generic_code'])
prescription_df['is_generic'] = prescription_df['product'] == 'AA'
prescription_df['unit_cost'] = prescription_df['ACT COST']/prescription_df['QUANTITY']

In [178]:
prescriptions_grouped = prescription_df.groupby(['GENERIC BNF', 'is_generic']).sum()[['QUANTITY','ACT COST']]
prescriptions_grouped = prescriptions_grouped.reset_index()
prescriptions_grouped['unit_cost'] = prescriptions_grouped['ACT COST']/prescriptions_grouped['QUANTITY']

In [179]:
prescriptions_generics = prescriptions_grouped[prescriptions_grouped['is_generic'] == True].reset_index().set_index('GENERIC BNF')
prescriptions_branded = prescriptions_grouped[prescriptions_grouped['is_generic'] == False].reset_index().set_index('GENERIC BNF')
joined_costs = prescriptions_generics.join(prescriptions_branded, lsuffix='_g', rsuffix='_b' )
expensive_b = joined_costs[joined_costs['unit_cost_b'] > joined_costs['unit_cost_g']].reindex()
expensive_b['excess'] = (expensive_b['unit_cost_b'] - expensive_b['unit_cost_g']) * expensive_b['QUANTITY_b']
top_ten = expensive_b.sort_values(by=['excess'], ascending=False).head(10).reset_index()
top_ten

,GENERIC BNF,index_g,is_generic_g,QUANTITY_g,ACT COST_g,unit_cost_g,index_b,is_generic_b,QUANTITY_b,ACT COST_b,unit_cost_b,excess
0,1404000H0AAAFAF,8222,True,15,115.13,7.675333,8221.0,False,3623137.0,30487353.94,8.414629,2.678570e+06
1,0302000C0AABFBF,1711,True,8103,55889.14,6.897339,1710.0,False,365270.0,3039102.68,8.320154,5.197116e+05
2,0408010A0AAABAB,3281,True,4244909,548945.55,0.129319,3280.0,False,694051.0,529957.61,0.763572,4.402039e+05
3,0408010A0AAACAC,3283,True,1453412,232497.45,0.159967,3282.0,False,270720.0,399461.31,1.475552,3.561551e+05
4,0408010H0AAAAAA,3450,True,4012112,1385921.71,0.345434,3449.0,False,382124.0,437391.13,1.144631,3.053923e+05
5,0703021Q0AAAAAA,5161,True,18012417,514929.18,0.028587,5160.0,False,11245857.0,600211.22,0.053372,2.787208e+05
6,0408010H0AAA1A1,3447,True,1238414,501925.12,0.405297,3446.0,False,165959.0,322839.36,1.945296,2.555767e+05
7,0106040M0AAAAAA,427,True,5541414,682393.09,0.123144,426.0,False,14497459.0,2021270.02,0.139422,2.359918e+05
8,0704050R0AAABAB,5331,True,196922,107695.44,0.546894,5330.0,False,36026.0,241458.40,6.702337,2.217560e+05
9,190700000AABABA,8411,True,321052,6490.52,0.020216,8410.0,False,29569473.0,775381.99,0.026222,1.775933e+05


In [180]:
"Total Saving £{}".format(expensive_b['excess'].sum())

'Total Saving £11347859.902464494'

In [181]:
bnf_lookup = prescription_df[['BNF CODE', 'BNF NAME']].copy().drop_duplicates().set_index('BNF CODE')

In [182]:
top_ten['name'] = top_ten['GENERIC BNF'].map(bnf_lookup.to_dict()['BNF NAME'])
summary = top_ten[['GENERIC BNF','name','excess']].copy()
top_ten_savings = summary['excess'].sum()
summary['excess'] = summary['excess']/1000
summary.rename({
    'GENERIC BNF':'BNF Code', 
    'name':'Pharmacetical Name',
    'excess':'Potential Saving £1,000s'
}, axis=1)

,BNF Code,Pharmacetical Name,"Potential Saving £1,000s"
0,1404000H0AAAFAF,Influenza_Vac Surf/Antgn Inact 0.5ml Pfs,2678.569753
1,0302000C0AABFBF,Beclomet Diprop_Inha 100mcg (200 D) CFF,519.711570
2,0408010A0AAABAB,Levetiracetam_Tab 500mg,440.203929
3,0408010A0AAACAC,Levetiracetam_Tab 1g,356.155138
4,0408010H0AAAAAA,Lamotrigine_Tab 100mg,305.392335
5,0703021Q0AAAAAA,Desogestrel_Tab 75mcg,278.720777
6,0408010H0AAA1A1,Lamotrigine_Tab 200mg,255.576723
7,0106040M0AAAAAA,Macrogol_Co Oral Pdr Sach S/F,235.991778
8,0704050R0AAABAB,Tadalafil_Tab 20mg,221.756000
9,190700000AABABA,Starch Maize Modified_Pdr G/F,177.593296


In [183]:
import locale
locale.setlocale(locale.LC_ALL, '')
"Total monthly savings from using top ten Generics £{}".format(
    locale.currency(top_ten_savings, symbol=False, grouping=True))


'Total monthly savings from using top ten Generics £5,469,671.30'